In [29]:
!pip install selenium
!pip install webdriver_manager

In [30]:
!sudo apt update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install
!google-chrome-stable --version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,083 B]
Fetched 236 kB in 3s (70.0 kB/s)
Reading package lists... Done
Building dependenc

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [32]:
!pip show webdriver_manager

Name: webdriver-manager
Version: 4.0.1
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: packaging, python-dotenv, requests
Required-by: 


In [33]:
import pandas as pd
import numpy as np

In [34]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.set_page_load_timeout(60)

In [35]:
data = "02/02/2023"
URL_DOL = f"https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-sistema-pregao-ptBR.asp?Data={data}&Mercadoria=DOL"
driver.get(URL_DOL)

In [36]:
vencimento = driver.find_element(By.ID, "MercadoFut0").text
vencimento

'VENCTO\nH23\nJ23\nK23\nM23\nN23\nQ23\nU23\nV23\nX23\nZ23\nF24\nG24\nJ24\nN24\nV24\nF25\nJ25\nN25\nV25\nF26'

In [37]:
vencimento = vencimento.split("\n")
vencimento = vencimento[1:]

In [38]:
vencimento

['H23',
 'J23',
 'K23',
 'M23',
 'N23',
 'Q23',
 'U23',
 'V23',
 'X23',
 'Z23',
 'F24',
 'G24',
 'J24',
 'N24',
 'V24',
 'F25',
 'J25',
 'N25',
 'V25',
 'F26']

In [39]:
tabela_vencimento = pd.DataFrame(vencimento, columns=["VENCTO"])
tabela_vencimento

,VENCTO
0,H23
1,J23
2,K23
3,M23
4,N23
5,Q23
6,U23
7,V23
8,X23
9,Z23


In [40]:
n_linhas = tabela_vencimento.shape[0]
n_linhas

20

In [41]:
# Captura dos dados do futuro de dólar
dados = driver.find_element(By.ID, "MercadoFut2").text
dados = dados.split()
dados[20:]

['4.963,500',
 '5.078,500',
 '5.026,117',
 '5.072,500',
 '5.056,5720',
 '55,9780-',
 '5.072,000',
 '5.075,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.089,7880',
 '56,2730-',
 '5.085,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.112,6420',
 '56,7030-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.145,3630',
 '57,0750-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.175,7530',
 '57,3620-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.205,5720',
 '57,1850-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.240,5020',
 '57,0370-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.267,3510',
 '57,3430-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.297,4430',
 '56,6620-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.324,4890',
 '56,0030-',
 '

In [42]:

dados = dados[19:] #remoção do cabeçalho
dados

['5.050,000',
 '4.963,500',
 '5.078,500',
 '5.026,117',
 '5.072,500',
 '5.056,5720',
 '55,9780-',
 '5.072,000',
 '5.075,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.100,000',
 '5.089,7880',
 '56,2730-',
 '5.085,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.112,6420',
 '56,7030-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.145,3630',
 '57,0750-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.175,7530',
 '57,3620-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.205,5720',
 '57,1850-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.240,5020',
 '57,0370-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.267,3510',
 '57,3430-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.297,4430',
 '56,6620-',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '0,000',
 '5.324,4890',
 

In [43]:
dados = np.array_split(dados, n_linhas)
dados

[array(['5.050,000', '4.963,500', '5.078,500', '5.026,117', '5.072,500',
        '5.056,5720', '55,9780-', '5.072,000', '5.075,000'], dtype='<U10'),
 array(['5.100,000', '5.100,000', '5.100,000', '5.100,000', '5.100,000',
        '5.089,7880', '56,2730-', '5.085,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.112,6420',
        '56,7030-', '0,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.145,3630',
        '57,0750-', '0,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.175,7530',
        '57,3620-', '0,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.205,5720',
        '57,1850-', '0,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.240,5020',
        '57,0370-', '0,000', '0,000'], dtype='<U10'),
 array(['0,000', '0,000', '0,000', '0,000', '0,000', '5.267,3510',
        '57,3430-', '0,000', '0,00

In [44]:
tabela_dados = pd.DataFrame(dados, columns=["PREÇO ABERT", "PREÇO MÍN.", "PREÇO MÁX.", "PREÇO MÉD.", "ÚLT. PREÇO", "AJUSTE", "VAR. PTOS.",	"ÚLT. OF. COMPRA",	"ÚLT. OF. VENDA"])
tabela_dados

,PREÇO ABERT,PREÇO MÍN.,PREÇO MÁX.,PREÇO MÉD.,ÚLT. PREÇO,AJUSTE,VAR. PTOS.,ÚLT. OF. COMPRA,ÚLT. OF. VENDA
0,"5.050,000","4.963,500","5.078,500","5.026,117","5.072,500","5.056,5720","55,9780-","5.072,000","5.075,000"
1,"5.100,000","5.100,000","5.100,000","5.100,000","5.100,000","5.089,7880","56,2730-","5.085,000","0,000"
2,"0,000","0,000","0,000","0,000","0,000","5.112,6420","56,7030-","0,000","0,000"
3,"0,000","0,000","0,000","0,000","0,000","5.145,3630","57,0750-","0,000","0,000"
4,"0,000","0,000","0,000","0,000","0,000","5.175,7530","57,3620-","0,000","0,000"
5,"0,000","0,000","0,000","0,000","0,000","5.205,5720","57,1850-","0,000","0,000"
6,"0,000","0,000","0,000","0,000","0,000","5.240,5020","57,0370-","0,000","0,000"
7,"0,000","0,000","0,000","0,000","0,000","5.267,3510","57,3430-","0,000","0,000"
8,"0,000","0,000","0,000","0,000","0,000","5.297,4430","56,6620-","0,000","0,000"
9,"0,000","0,000","0,000","0,000","0,000","5.324,4890","56,0030-","0,000","0,000"


In [45]:
# Consolidar tudo em uma tabela
df_dol = pd.merge(tabela_vencimento, tabela_dados, left_index=True, right_index=True, how="inner")
df_dol

,VENCTO,PREÇO ABERT,PREÇO MÍN.,PREÇO MÁX.,PREÇO MÉD.,ÚLT. PREÇO,AJUSTE,VAR. PTOS.,ÚLT. OF. COMPRA,ÚLT. OF. VENDA
0,H23,"5.050,000","4.963,500","5.078,500","5.026,117","5.072,500","5.056,5720","55,9780-","5.072,000","5.075,000"
1,J23,"5.100,000","5.100,000","5.100,000","5.100,000","5.100,000","5.089,7880","56,2730-","5.085,000","0,000"
2,K23,"0,000","0,000","0,000","0,000","0,000","5.112,6420","56,7030-","0,000","0,000"
3,M23,"0,000","0,000","0,000","0,000","0,000","5.145,3630","57,0750-","0,000","0,000"
4,N23,"0,000","0,000","0,000","0,000","0,000","5.175,7530","57,3620-","0,000","0,000"
5,Q23,"0,000","0,000","0,000","0,000","0,000","5.205,5720","57,1850-","0,000","0,000"
6,U23,"0,000","0,000","0,000","0,000","0,000","5.240,5020","57,0370-","0,000","0,000"
7,V23,"0,000","0,000","0,000","0,000","0,000","5.267,3510","57,3430-","0,000","0,000"
8,X23,"0,000","0,000","0,000","0,000","0,000","5.297,4430","56,6620-","0,000","0,000"
9,Z23,"0,000","0,000","0,000","0,000","0,000","5.324,4890","56,0030-","0,000","0,000"


In [46]:
from datetime import date, timedelta

# Definindo o ano para o qual o código será gerado
year = 2023

# Feriados em 2023
holidays = [
    "01/01/2023", "20/02/2023", "21/02/2023", "22/02/2023", "07/04/2023",
    "21/04/2023", "01/05/2023", "08/06/2023", "07/09/2023", "12/10/2023",
    "02/11/2023", "15/11/2023", "25/12/2023", "29/12/2023"
]

# Convertendo strings de feriados para objetos date
holidays = [date.fromisoformat(f"{day[-4:]}-{day[3:5]}-{day[0:2]}") for day in holidays]

# Gerar todas as datas do ano, excluindo sábados, domingos e feriados
start_date = date(year, 1, 1)
end_date = date(year, 12, 31)
all_dates = []

current_date = start_date
while current_date <= end_date:
    if current_date.weekday() < 5 and current_date not in holidays:  # 0-4 são dias da semana (Seg-Sex)
        all_dates.append(current_date.strftime("%d/%m/%Y"))
    current_date += timedelta(days=1)

In [47]:
# Criando um objeto ExcelWriter
with pd.ExcelWriter('dados_dol_2023_por_dia.xlsx', engine='openpyxl') as writer:
    for data in all_dates:
        print(data)
        URL_DOL = f"https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-sistema-pregao-ptBR.asp?Data={data}&Mercadoria=DOL"
        driver.get(URL_DOL)

        vencimento = driver.find_element(By.ID, "MercadoFut0").text.split("\n")[1:]
        tabela_vencimento = pd.DataFrame(vencimento, columns=["VENCTO"])
        n_linhas = tabela_vencimento.shape[0]

        dados = driver.find_element(By.ID, "MercadoFut2").text.split()[19:]
        dados = np.array_split(dados, n_linhas)
        tabela_dados = pd.DataFrame(dados, columns=["PREÇO ABERT", "PREÇO MÍN.", "PREÇO MÁX.", "PREÇO MÉD.", "ÚLT. PREÇO", "AJUSTE", "VAR. PTOS.", "ÚLT. OF. COMPRA", "ÚLT. OF. VENDA"])

        # Tratamento de dados
        for coluna in tabela_dados.columns:
            if coluna != 'VAR. PTOS.':
                # Trata os valores monetários, convertendo para o formato desejado
                tabela_dados[coluna] = tabela_dados[coluna].str.replace(',', '').str.replace('.', '').astype(str).str.slice(0, 1) + '.' + tabela_dados[coluna].str.replace(',', '').str.replace('.', '').astype(str).str.slice(1,)
                tabela_dados[coluna] = tabela_dados[coluna].astype(float)
            else:
                # Trata a coluna 'VAR. PTOS.' convertendo valores e ajustando sinais
                tabela_dados[coluna] = tabela_dados[coluna].apply(lambda x: float(x.replace(',', '.').rstrip('+')) if '+' in x else -float(x.replace(',', '.').rstrip('-')))

        df_dol = pd.merge(tabela_vencimento, tabela_dados, left_index=True, right_index=True, how="inner")

        # Salvando o DataFrame na aba correspondente à data
        df_dol.to_excel(writer, sheet_name=data.replace('/','_'), index=False)

driver.quit()

print("Dados salvos com sucesso em abas separadas por data!")

02/01/2023
03/01/2023
04/01/2023
05/01/2023
06/01/2023
09/01/2023
10/01/2023
11/01/2023
12/01/2023
13/01/2023
16/01/2023
17/01/2023
18/01/2023
19/01/2023
20/01/2023
23/01/2023
24/01/2023
25/01/2023
26/01/2023
27/01/2023
30/01/2023
31/01/2023
01/02/2023
02/02/2023
03/02/2023
06/02/2023
07/02/2023
08/02/2023
09/02/2023
10/02/2023
13/02/2023
14/02/2023
15/02/2023
16/02/2023
17/02/2023
23/02/2023
24/02/2023
27/02/2023
28/02/2023
01/03/2023
02/03/2023
03/03/2023
06/03/2023
07/03/2023
08/03/2023
09/03/2023
10/03/2023
13/03/2023
14/03/2023
15/03/2023
16/03/2023
17/03/2023
20/03/2023
21/03/2023
22/03/2023
23/03/2023
24/03/2023
27/03/2023
28/03/2023
29/03/2023
30/03/2023
31/03/2023
03/04/2023
04/04/2023
05/04/2023
06/04/2023
10/04/2023
11/04/2023
12/04/2023
13/04/2023
14/04/2023
17/04/2023
18/04/2023
19/04/2023
20/04/2023
24/04/2023
25/04/2023
26/04/2023
27/04/2023
28/04/2023
02/05/2023
03/05/2023
04/05/2023
05/05/2023
08/05/2023
09/05/2023
10/05/2023
11/05/2023
12/05/2023
15/05/2023
16/05/2023

In [48]:
# Carregar a planilha
caminho_arquivo = 'dados_dol_2023_por_dia.xlsx'
dados = pd.read_excel(caminho_arquivo, sheet_name='02_01_2023', engine='openpyxl')

# Exibindo as primeiras linhas do DataFrame
print("Primeiras linhas dos dados carregados:")
print(dados.head())

# Exibindo informações básicas sobre o DataFrame
print("\nInformações do DataFrame:")
print(dados.info())


Primeiras linhas dos dados carregados:
  VENCTO  PREÇO ABERT  PREÇO MÍN.  PREÇO MÁX.  PREÇO MÉD.  ÚLT. PREÇO  \
0    F23        0.000       0.000         0.0    0.000000         0.0   
1    G23        5.323       5.321         5.4    5.380172         5.4   
2    H23        0.000       0.000         0.0    0.000000         0.0   
3    J23        0.000       0.000         0.0    0.000000         0.0   
4    K23        0.000       0.000         0.0    0.000000         0.0   

     AJUSTE  VAR. PTOS.  ÚLT. OF. COMPRA  ÚLT. OF. VENDA  
0  5.217700       0.000            0.000           0.000  
1  5.390280      66.312            5.399           5.400  
2  5.416326      66.957            5.406           5.454  
3  5.451615      67.263            0.000           0.000  
4  5.476124      67.705            0.000           0.000  

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column           Non-Null Count  

'5.399.000'